In [301]:
from policy import *
import numpy as np
import random
import copy

newGame = [1, 3, 5, 7]
state = [1, 3, 5, 7]
visits = {} #keeps track of how many times each state is visited

win = (0, 0, 0, 0)

# Reward values for win and loss conditions
win_reward = 100
lose_reward = -50

# episode length
maxrun = 100

In [302]:
def Initialize():
    global policy
    global value
    global q
    global visits
    policy = copy.deepcopy(Policy_Template) #replaced .copy with a deepcopy
    value = copy.deepcopy(Value_Template)
    q = copy.deepcopy(Q_Template)
    visits = copy.deepcopy(Visits_Template)

    return


def New_Game():
    global state
    state = newGame.copy()

    return


In [303]:
"""
A function for setting the policy for a given state
r1: Number of sticks in row 1
r2: Number of sticks in row 2
r3: Number of sticks in row 3
r4: Number of sticks in row 4
count: Total number of sticks left
"""
def setPolicy(r1, r2, r3, r4, count):
    policy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(r1):
        policy[0 + i] = 1.0/count
    for i in range(r2):
        policy[1 + i] = 1.0/count
    for i in range(r3):
        policy[4 + i] = 1.0/count
    for i in range(r4):
        policy[9 + i] = 1.0/count
    return policy

"""
Generates a new policy for NIM
"""
def newPolicy():
    for r1 in range(2):
        for r2 in range(4):
            for r3 in range(6):
                for r4 in range(8):
                    print((r1, r2, r3, r4), end='')
                    print(" : ", end='')
                    print(setPolicy(r1, r2, r3, r4, r1 + r2 + r3 + r4), end='')
                    print(",")

In [304]:
"""
Observes the current state space, and makes a random legal action
"""
def randomAction():
    while True: # it randomly chooses actions until it finds one that is legal
        action = random.randint(0, 15)
        if Policy_Template[tuple(state)][action] != 0:
            break
    #print(Policy_Template[tuple(state)]) # here in case we need more debugging
    takeAction(Actions[action])

"""
Takes in a row (1, 2, 3 or 4) and removes the specified number of sticks.
If it needs to remove more sticks than what the row contains, it terminates the program
"""
def takeAction(actionTuple):
    row, nSticks = actionTuple
    if state[row - 1] < nSticks: # simple error check
        print("Attempting to remove more sticks than possible, this shouldn't be possible!")
        quit()
    state[row - 1] -= nSticks # removes specified stick #
    updateInfo()

"""
Updates various information when the state space changes. 
Currently only updates Visits, should probably update values or Q or smthn idk
"""
def updateInfo():
    if tuple(state) == win:
        return
    visits[tuple(state)]# += 1
    # value?


"""
Plays the game by taking random legal moves
"""
def playRandomly():
    New_Game()
    print(state)
    while (tuple(state) != win):
        randomAction()
        print("action taken")
        print(state)



In [305]:
def main():
    Initialize()
    playRandomly()

if __name__ == "__main__":
    main()

[1, 3, 5, 7]
action taken
[1, 3, 4, 7]
action taken
[1, 3, 4, 1]
action taken
[1, 3, 0, 1]
action taken
[1, 3, 0, 0]
action taken
[0, 3, 0, 0]
action taken
[0, 2, 0, 0]
action taken
[0, 0, 0, 0]
